In [1]:
from flask import Flask,render_template,request,redirect,session
import requests
import pandas as pd
import datetime as dt
import numpy as np
import pickle
from scipy import spatial
import itertools
from scipy import signal
from collections import Counter
import simplejson as json
import urllib2 
import urllib

In [25]:
url = 'https://www.quandl.com/api/v3/datasets/WIKI/AAPL.jso?api_key=khszD7FbY99W9haQ6ZZr&start_date=2016-08-20'
r = requests.get(url, auth=('user', 'pass'))
print r.status_code
#print r.headers['content-type']
#r.encoding

blerg = 0
if(r.status_code == 200):
    blerg = 1
print blerg

406
0


In [3]:
text = r.text
print text
json.loads(text)
#my_loads = json.loads(text)
#print my_loads

{"dataset":{"id":9775409,"dataset_code":"AAPL","database_code":"WIKI","name":"Apple Inc (AAPL) Prices, Dividends, Splits and Trading Volume","description":"End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Apple Inc. (AAPL). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (\u003ca href=\"http://www.crsp.com/products/documentation/crsp-calculations\" rel=\"nofollow\" target=\"blank\"\u003ewww.crsp.com/products/documentation/crsp-calculations\u003c/a\u003e)\r\n\r\n\u003cp\u003eThis data is in the public domain. You may copy, distribute, disseminate or include the data in other products for commercial and/or noncommercial purposes.\u003c/p\u003e\r\n\u003cp\u003eThis data is part of Quandl's Wiki initiative to get financial data permanently into the public domain. Quandl relies on users like you to flag errors and provide data where d

{u'dataset': {u'collapse': None,
  u'column_index': None,
  u'column_names': [u'Date',
   u'Open',
   u'High',
   u'Low',
   u'Close',
   u'Volume',
   u'Ex-Dividend',
   u'Split Ratio',
   u'Adj. Open',
   u'Adj. High',
   u'Adj. Low',
   u'Adj. Close',
   u'Adj. Volume'],
  u'data': [[u'2016-08-26',
    107.41,
    107.95,
    106.31,
    106.94,
    27534384.0,
    0.0,
    1.0,
    107.41,
    107.95,
    106.31,
    106.94,
    27534384.0],
   [u'2016-08-25',
    107.39,
    107.88,
    106.68,
    107.57,
    24840787.0,
    0.0,
    1.0,
    107.39,
    107.88,
    106.68,
    107.57,
    24840787.0],
   [u'2016-08-24',
    108.565,
    108.75,
    107.68,
    108.03,
    22807722.0,
    0.0,
    1.0,
    108.565,
    108.75,
    107.68,
    108.03,
    22807722.0],
   [u'2016-08-23',
    108.59,
    109.32,
    108.53,
    108.85,
    19615651.0,
    0.0,
    1.0,
    108.59,
    109.32,
    108.53,
    108.85,
    19615651.0],
   [u'2016-08-22',
    108.86,
    109.1,
    107.

In [4]:
mydata = r.json()
print mydata

{u'dataset': {u'dataset_code': u'AAPL', u'column_names': [u'Date', u'Open', u'High', u'Low', u'Close', u'Volume', u'Ex-Dividend', u'Split Ratio', u'Adj. Open', u'Adj. High', u'Adj. Low', u'Adj. Close', u'Adj. Volume'], u'newest_available_date': u'2016-08-26', u'description': u'End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Apple Inc. (AAPL). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (<a href="http://www.crsp.com/products/documentation/crsp-calculations" rel="nofollow" target="blank">www.crsp.com/products/documentation/crsp-calculations</a>)\r\n\r\n<p>This data is in the public domain. You may copy, distribute, disseminate or include the data in other products for commercial and/or noncommercial purposes.</p>\r\n<p>This data is part of Quandl\'s Wiki initiative to get financial data permanently into the public domain. Quan

In [5]:
my_df = pd.read_json(text)

In [6]:
my_df

,dataset
collapse,None
column_index,None
column_names,"[Date, Open, High, Low, Close, Volume, Ex-Divi..."
data,"[[2016-08-26, 107.41, 107.95, 106.31, 106.94, ..."
database_code,WIKI
database_id,4922
dataset_code,AAPL
description,"End of day open, high, low, close and volume, ..."
end_date,2016-08-26
frequency,daily


In [7]:
print my_df.dataset.id
print my_df.dataset.dataset_code
print my_df.dataset.column_names
print my_df['dataset']['column_names']
print my_df['dataset']['data']

9775409
AAPL
[u'Date', u'Open', u'High', u'Low', u'Close', u'Volume', u'Ex-Dividend', u'Split Ratio', u'Adj. Open', u'Adj. High', u'Adj. Low', u'Adj. Close', u'Adj. Volume']
[u'Date', u'Open', u'High', u'Low', u'Close', u'Volume', u'Ex-Dividend', u'Split Ratio', u'Adj. Open', u'Adj. High', u'Adj. Low', u'Adj. Close', u'Adj. Volume']
[[u'2016-08-26', 107.41, 107.95, 106.31, 106.94, 27534384.0, 0.0, 1.0, 107.41, 107.95, 106.31, 106.94, 27534384.0], [u'2016-08-25', 107.39, 107.88, 106.68, 107.57, 24840787.0, 0.0, 1.0, 107.39, 107.88, 106.68, 107.57, 24840787.0], [u'2016-08-24', 108.565, 108.75, 107.68, 108.03, 22807722.0, 0.0, 1.0, 108.565, 108.75, 107.68, 108.03, 22807722.0], [u'2016-08-23', 108.59, 109.32, 108.53, 108.85, 19615651.0, 0.0, 1.0, 108.59, 109.32, 108.53, 108.85, 19615651.0], [u'2016-08-22', 108.86, 109.1, 107.85, 108.51, 25223957.0, 0.0, 1.0, 108.86, 109.1, 107.85, 108.51, 25223957.0]]


In [158]:
print my_df.dataset.description
print my_df.dataset.column_names
print my_df.dataset.frequency
print my_df['dataset']['data']

End of day open, high, low, close and volume, dividends and splits, and split/dividend adjusted open, high, low close and volume for Apple Inc. (AAPL). Ex-Dividend is non-zero on ex-dividend dates. Split Ratio is 1 on non-split dates. Adjusted prices are calculated per CRSP (<a href="http://www.crsp.com/products/documentation/crsp-calculations" rel="nofollow" target="blank">www.crsp.com/products/documentation/crsp-calculations</a>)

<p>This data is in the public domain. You may copy, distribute, disseminate or include the data in other products for commercial and/or noncommercial purposes.</p>
<p>This data is part of Quandl's Wiki initiative to get financial data permanently into the public domain. Quandl relies on users like you to flag errors and provide data where data is wrong or missing. Get involved: <a href="mailto:connect@quandl.com" rel="nofollow" target="blank">connect@quandl.com</a>
[u'Date', u'Open', u'High', u'Low', u'Close', u'Volume', u'Ex-Dividend', u'Split Ratio', u'Ad

In [8]:
print my_df.dataset['column_names']
print my_df['dataset']['data']
print my_df['dataset']['data'][1]
print len(my_df['dataset']['data'][:])
length = len(my_df['dataset']['data'][:])

c_names = my_df.dataset['column_names']

datelist = np.arange('2004-02-01', '2005-02-01', dtype='datetime64[D]')

datelist = datelist[0:length]

open_price = np.arange(length-1, dtype='float64')
high_price = np.arange(length-1, dtype='float64')
low_price = np.arange(length-1, dtype='float64')
close_price = np.arange(length-1, dtype='float64')

print datelist

for i in range(length-1):
    datelist[i] = my_df['dataset']['data'][i][0]
    open_price[i] = my_df['dataset']['data'][i][1]
    high_price[i] = my_df['dataset']['data'][i][2]
    low_price[i] = my_df['dataset']['data'][i][3]
    close_price[i] = my_df['dataset']['data'][i][4]
    

print datelist
print open_price
print high_price
print low_price
print close_price
    
#my_new_df = DataFrame({'a': column1,
#                'b': column2,
#                'c': column3},
#                 index=c_names)


[u'Date', u'Open', u'High', u'Low', u'Close', u'Volume', u'Ex-Dividend', u'Split Ratio', u'Adj. Open', u'Adj. High', u'Adj. Low', u'Adj. Close', u'Adj. Volume']
[[u'2016-08-26', 107.41, 107.95, 106.31, 106.94, 27534384.0, 0.0, 1.0, 107.41, 107.95, 106.31, 106.94, 27534384.0], [u'2016-08-25', 107.39, 107.88, 106.68, 107.57, 24840787.0, 0.0, 1.0, 107.39, 107.88, 106.68, 107.57, 24840787.0], [u'2016-08-24', 108.565, 108.75, 107.68, 108.03, 22807722.0, 0.0, 1.0, 108.565, 108.75, 107.68, 108.03, 22807722.0], [u'2016-08-23', 108.59, 109.32, 108.53, 108.85, 19615651.0, 0.0, 1.0, 108.59, 109.32, 108.53, 108.85, 19615651.0], [u'2016-08-22', 108.86, 109.1, 107.85, 108.51, 25223957.0, 0.0, 1.0, 108.86, 109.1, 107.85, 108.51, 25223957.0]]
[u'2016-08-25', 107.39, 107.88, 106.68, 107.57, 24840787.0, 0.0, 1.0, 107.39, 107.88, 106.68, 107.57, 24840787.0]
5
['2004-02-01' '2004-02-02' '2004-02-03' '2004-02-04' '2004-02-05']
['2016-08-26' '2016-08-25' '2016-08-24' '2016-08-23' '2004-02-05']
[ 107.41   10

In [13]:
from bokeh.layouts import gridplot
from bokeh.plotting import figure, show, output_file
from bokeh.embed import components 

TOOLS = "pan,wheel_zoom,box_zoom,reset"

plot = figure(tools=TOOLS,
              title='AAPL Data from Quandle WIKI set',
              x_axis_label='date',
              x_axis_type='datetime')

plot.line(datelist, open_price, color='blue', legend = 'opening price')

show(plot)

#script, div = components(plot)
#return render_template('graph.html', script=script, div=div)